<a href="https://colab.research.google.com/github/itchy-squid/AdventOfCode/blob/2025-2/2025/2025.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
fread = lambda path: open(path,'r').read()

In [ ]:
#@title Day 1: Secret Entrance
import math

inputs = fread('25-1.txt').splitlines()

def rotate(pos, d, rotation, part1, part2):
  flip = lambda p : [p%100,(100-p)%100][d=='R']
  pos = flip(pos) + rotation
  return (flip(pos), part1 + (flip(pos) == 0), part2 + math.floor(pos / 100))

acc = (50, 0, 0)
[acc := rotate(acc[0], i[0],int(i[1:]), acc[1], acc[2]) for i in inputs]

print(acc[1])
print(acc[2])

In [33]:
#@title Day 2: Gift Shop
import math, re

inputs = fread('input.txt')
inputs = [list(map(int,s.split('-'))) for s in inputs.split(',')]

def bads(lower, upper, pattern):
  all = [i for i in range(lower, upper+1)]
  some = [i for i in all if re.search(pattern, str(i))]
  return sum([s for s in some if s >= lower and s <= upper])

print(sum([bads(*l,r'^([0-9]+)\1$') for l in inputs]))
print(sum([bads(*l,r'^([0-9]+)\1+$') for l in inputs]))

5398419778
15704845910
